This is for making datasets to train.

In [ ]:
# make csv file and header for data
import csv
import os
import numpy as np
num_coords = len(results.pose_landmarks.landmark)
num_coords
landmarks = ['class']

# Create a list of the landmark names
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val),
                  'z{}'.format(val), 'v{}'.format(val)]

# Make CSV file with header
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(
        f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)


In [ ]:
# Activity Class
class_name = "activity name"


In [ ]:
# Get the coordinates from every images
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import glob
import csv
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# For Image Processing
data = []
img_dir = "directory path for images"
data_path = os.path.join(img_dir, '*g')
files = glob.glob(data_path)
BG_COLOR = (192, 192, 192)  # gray

with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=1,
        enable_segmentation=True,
        min_detection_confidence=0.1) as pose:

    for idx, file in enumerate(files):
        # Input Blazepose
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        print(idx, file)
        try:
            for landmark in results.pose_landmarks.landmark:
                x = int(landmark.x * image.shape[1])
                y = int(landmark.y * image.shape[0])
                annotated_image = image.copy()
                condition = np.stack(
                    (results.segmentation_mask,) * 3, axis=-1) > 0.1
                bg_image = np.zeros(image.shape, dtype=np.uint8)
                bg_image[:] = BG_COLOR
                annotated_image = np.where(
                    condition, annotated_image, bg_image)
                mp_drawing.draw_landmarks(
                    annotated_image,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            # Output Blazepose
            data = results.pose_landmarks.landmark
            pose_row = list(np.array(
                [[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in data]).flatten())
            row = pose_row
            row.insert(0, class_name)

            # Write to CSV file
            with open('coords1.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(
                    f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
        except:
            # If caught an error, skip the image
            print("Error")
            continue
